# Final Project: Functional Programming

**Laura Belizón Merchán** - 100452273  
**Jorge Lázaro Ruiz** - 100452172  
*Degree in Applied Mathematics and Computation*

## Brief summary

*Pending...*

In [1]:
-- Necessary imports
import Data.Maybe (fromJust, fromMaybe)
import Data.List (intercalate, find)
import Debug.Trace

## Types

> 1. Types to represent all the former concepts. An exhaustive type definition is required, use the most appropriate way (type, data, newtype) for each of them.

An agenda is described as a list of events, so it makes sense to center the type definitions around the characteristics of an event.

For this, we can create the different types of descriptors defined and then use them for the constructor of an event.

In [2]:
type Name = String
data Type = Personal | Health | Work | Management
data Date = Date {day :: Int, month :: String, year :: Int, workday :: Bool}
data Time = Time {start :: Maybe (Int, Int), end :: Maybe (Int, Int), duration :: Maybe Int}
newtype Participants = Participants [Name]
data Repetitions = Punctual | Daily Int | Weekday Int | Weekly Int

* `Name` is just an alias for a `String`, as there are no spectial requirements for it.
* `Type` and `Repetitions` need to be instances of class `Show` if we want to be able to print them, and since they can only take a set of values we must define them as a `data`.
* `Date` and `Time` require a constructor with multiple parameters, hence we use `data` to define them. They also have to be instances of class `Show` so we can pretty-print them.
* `Participants` could be defined as an alias for `[Name]` but in order to avoid confusion with just any other list of strings, we decided to make it into a `newtype` to distinguish it and prevent unintended operations with other lists of strings.

Because we will need to display this to the user, we declare each type as a `Show` instance and define `show` for each one.

Showing the `Time` type is a bit more complicated, as it must take into account all possible missing fields. It is also required that duration is always displayed, so we need to compute it from the start and end times whenever it is not given explicitly. The same applies when either the start or end time is missing, as we need to compute it from the other one and the duration. Insufficient data is handled as an error.

In [3]:
instance Show Type where
    show Personal = "Personal"
    show Health = "Health"
    show Work = "Work"
    show Management = "Management"

instance Show Date where
    show (Date day month year workday) = show day ++ " " ++ month ++ " " ++ show year ++ " " ++
        if workday then "(Workday)" else ""

leadingZero :: Int -> String
leadingZero n = if n < 10 then "0" ++ show n else show n

instance Show Time where
    show (Time Nothing Nothing Nothing) = error "Invalid time"
    show (Time (Just (startHour, startMinute)) Nothing Nothing) = error "Invalid time"
    show (Time Nothing (Just (endHour, endMinute)) Nothing) = error "Invalid time"
    show (Time Nothing Nothing (Just duration)) = "(" ++ show duration ++ " minutes)"
    show (Time (Just (startHour, startMinute)) (Just (endHour, endMinute)) Nothing) =
        leadingZero startHour ++ ":" ++ leadingZero startMinute ++ " - " ++
        leadingZero endHour ++ ":" ++ leadingZero endMinute ++
        " (" ++ show ((endHour * 60 + endMinute) - (startHour * 60 + startMinute)) ++ " minutes)"
    show (Time (Just (startHour, startMinute)) Nothing (Just duration)) =
        leadingZero startHour ++ ":" ++ leadingZero startMinute ++ " - " ++ 
        leadingZero ((startHour * 60 + startMinute + duration) `div` 60) ++ ":" ++
        leadingZero ((startHour * 60 + startMinute + duration) `mod` 60) ++
        " (" ++ show duration ++ " minutes)"
    show (Time Nothing (Just (endHour, endMinute)) (Just duration)) =
        leadingZero ((endHour * 60 + endMinute - duration) `div` 60) ++ ":" ++
        leadingZero ((endHour * 60 + endMinute - duration) `mod` 60) ++ " - " ++
        leadingZero endHour ++ ":" ++ leadingZero endMinute ++
        " (" ++ show duration ++ " minutes)"
    show (Time (Just (startHour, startMinute)) (Just (endHour, endMinute)) (Just duration)) =
        leadingZero startHour ++ ":" ++ leadingZero startMinute ++ " - " ++
        leadingZero endHour ++ ":" ++ leadingZero endMinute ++
        " (" ++ show duration ++ " minutes)"

instance Show Participants where
    show (Participants names) = intercalate ", " names

instance Show Repetitions where
    show Punctual = "Punctual"
    show (Daily n) = "Daily " ++ show n
    show (Weekday n) = "Weekday " ++ show n
    show (Weekly n) = "Weekly " ++ show n

Because we will need to compare descriptors, we declare each one as an `Eq` instance and define `==` for each one.

The behavior of comparing `Time`s is a bit more complex, as it needs to account for all possible combinations of fields.

In [4]:
instance Eq Type where
    Personal == Personal = True
    Health == Health = True
    Work == Work = True
    Management == Management = True
    _ == _ = False

monthToInt :: String -> Int
monthToInt month = fromJust $ lookup month (zip months [0..13])
    where months = ["Error", "January", "February", "March", "April", "May", "June", "July", "August", "September",
                    "October", "November", "December", "Error"]

instance Eq Date where
    (Date day1 month1 year1 _) == (Date day2 month2 year2 _) = 
        day1 == day2 && month1 == month2 && year1 == year2

instance Eq Time where
    (Time Nothing Nothing Nothing) == (Time Nothing Nothing Nothing) = True
    (Time (Just (startHour1, startMinute1)) Nothing Nothing) == (Time (Just (startHour2, startMinute2)) Nothing Nothing) =
        startHour1 == startHour2 && startMinute1 == startMinute2
    (Time Nothing (Just (endHour1, endMinute1)) Nothing) == (Time Nothing (Just (endHour2, endMinute2)) Nothing) =
        endHour1 == endHour2 && endMinute1 == endMinute2
    (Time Nothing Nothing (Just duration1)) == (Time Nothing Nothing (Just duration2)) = duration1 == duration2
    (Time (Just (startHour1, startMinute1)) (Just (endHour1, endMinute1)) Nothing) ==
        (Time (Just (startHour2, startMinute2)) (Just (endHour2, endMinute2)) Nothing) =
        startHour1 == startHour2 && startMinute1 == startMinute2 &&
        endHour1 == endHour2 && endMinute1 == endMinute2
    (Time (Just (startHour1, startMinute1)) Nothing (Just duration1)) ==
        (Time (Just (startHour2, startMinute2)) Nothing (Just duration2)) =
        startHour1 == startHour2 && startMinute1 == startMinute2 && duration1 == duration2
    (Time Nothing (Just (endHour1, endMinute1)) (Just duration1)) ==
        (Time Nothing (Just (endHour2, endMinute2)) (Just duration2)) =
        endHour1 == endHour2 && endMinute1 == endMinute2 && duration1 == duration2
    (Time (Just (startHour1, startMinute1)) (Just (endHour1, endMinute1)) (Just duration1)) ==
        (Time (Just (startHour2, startMinute2)) (Just (endHour2, endMinute2)) (Just duration2)) =
        startHour1 == startHour2 && startMinute1 == startMinute2 &&
        endHour1 == endHour2 && endMinute1 == endMinute2 && duration1 == duration2

instance Ord Date where
    (Date d1 m1 y1 _) > (Date d2 m2 y2 _) = [y1, monthToInt m1, d1] > [y2, monthToInt m2, d2]
    (Date d1 m1 y1 _) < (Date d2 m2 y2 _) = [y1, monthToInt m1, d1] < [y2, monthToInt m2, d2]
    (Date d1 m1 y1 _) >= (Date d2 m2 y2 _) = [y1, monthToInt m1, d1] >= [y2, monthToInt m2, d2]
    (Date d1 m1 y1 _) <= (Date d2 m2 y2 _) = [y1, monthToInt m1, d1] <= [y2, monthToInt m2, d2]

instance Ord Time where
    (Time (Just (startHour1, startMinute1)) _ _) > (Time (Just (startHour2, startMinute2)) _ _) =
        startHour1 >= startHour2 && startMinute1 > startMinute2
    (Time (Just (startHour1, startMinute1)) _ _) < (Time (Just (startHour2, startMinute2)) _ _) =
        startHour1 <= startHour2 && startMinute1 < startMinute2
    (Time (Just (startHour1, startMinute1)) _ _) >= (Time (Just (startHour2, startMinute2)) _ _) =
        startHour1 >= startHour2 && startMinute1 >= startMinute2
    (Time (Just (startHour1, startMinute1)) _ _) <= (Time (Just (startHour2, startMinute2)) _ _) =
        startHour1 <= startHour2 && startMinute1 <= startMinute2

instance Eq Participants where
    (Participants names1) == (Participants names2) = names1 == names2

instance Eq Repetitions where
    Punctual == Punctual = True
    (Daily n1) == (Daily n2) = n1 == n2
    (Weekday n1) == (Weekday n2) = n1 == n2
    (Weekly n1) == (Weekly n2) = n1 == n2
    _ == _ = False

In [5]:
instance Enum Date where
    succ dt@(Date d m y w)
        | not (rightMonth dt) || not (rightDay dt) = Date 0 "Error" 0 w
        | not (rightDay (Date (d+1) m y w)) && not (rightMonth (Date d (intToMonth (monthToInt m + 1)) y w)) = Date 1 "January" (y+1) w
        | not (rightDay (Date (d+1) m y w)) = Date 1 (intToMonth (monthToInt m + 1)) y w
        | otherwise = Date (d+1) m y w
    pred dt@(Date d m y w)
        | not (rightMonth dt) || not (rightDay dt) = Date 0 "Error" 0 w
        | not (rightDay (Date (d-1) m y w)) && not (rightMonth (Date d (intToMonth (monthToInt m - 1)) y w)) = Date 31 "December" (y-1) w
        | not (rightDay (Date (d-1) m y w)) = if m == "March" && isleap dt then Date 29 "February" y w else if m == "March" then Date 28 "February" y w else if m `elem` m31 then Date 31 (intToMonth (monthToInt m - 1)) y w else Date 30 (intToMonth (monthToInt m - 1)) y w
        | otherwise = Date (d-1) m y w
        where m31 = ["February", "April", "June", "August", "September", "November"]

rightMonth :: Date -> Bool
rightMonth (Date d m y w) = m `elem` months
    where months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

rightDay :: Date -> Bool
rightDay dt@(Date d m y w)
    | d <= 0 = False
    | m == "February" && d <= 28 = True
    | m == "February" && not (isleap dt) && d >= 29 = False
    | elem m m31 && d <= 31 = True
    | d <= 30 = True
    | otherwise = False
    where m31 = ["January", "March", "May", "July", "August", "October", "December"]

isleap :: Date -> Bool
isleap (Date d m y w)
    | mod y 400 == 0 = True
    | mod y 100 == 0 = False
    | mod y 4 == 0 = True
    | otherwise = False

intToMonth :: Int -> String
intToMonth month = fromJust $ lookup month (zip [0..13] months)
    where months = ["Error", "January", "February", "March", "April", "May", "June", "July", "August", "September",
                    "October", "November", "December", "Error"]

Because we will need to compare descriptors, we declare each one as an `Ord` instance and define the comparison operators.

In [6]:
instance Ord Date where
    (Date d1 m1 y1 _) > (Date d2 m2 y2 _) = [y1, monthToInt m1, d1] > [y2, monthToInt m2, d2]
    (Date d1 m1 y1 _) < (Date d2 m2 y2 _) = [y1, monthToInt m1, d1] < [y2, monthToInt m2, d2]
    (Date d1 m1 y1 _) >= (Date d2 m2 y2 _) = [y1, monthToInt m1, d1] >= [y2, monthToInt m2, d2]
    (Date d1 m1 y1 _) <= (Date d2 m2 y2 _) = [y1, monthToInt m1, d1] <= [y2, monthToInt m2, d2]

instance Ord Time where
    (Time (Just (startHour1, startMinute1)) _ _) > (Time (Just (startHour2, startMinute2)) _ _) =
        startHour1 >= startHour2 && startMinute1 > startMinute2
    (Time (Just (startHour1, startMinute1)) _ _) < (Time (Just (startHour2, startMinute2)) _ _) =
        startHour1 <= startHour2 && startMinute1 < startMinute2
    (Time (Just (startHour1, startMinute1)) _ _) >= (Time (Just (startHour2, startMinute2)) _ _) =
        startHour1 >= startHour2 && startMinute1 >= startMinute2
    (Time (Just (startHour1, startMinute1)) _ _) <= (Time (Just (startHour2, startMinute2)) _ _) =
        startHour1 <= startHour2 && startMinute1 <= startMinute2

With all the types defined, all we need to do is create the type `Descriptor`, with a constructor for each field.  
After that, we can define `Event` as a list of `Descriptor`s and `Agenda` as a list of `Event`s.

In [7]:
data Descriptor =
    NameDescriptor Name
    | TypeDescriptor Type
    | DateDescriptor Date
    | TimeDescriptor Time
    | ParticipantsDescriptor Participants
    | RepetitionsDescriptor Repetitions
    deriving Show

type Event = [Descriptor]
type Agenda = [Event]

In [8]:
instance Eq Descriptor where
    (NameDescriptor name1) == (NameDescriptor name2) = name1 == name2
    (TypeDescriptor type1) == (TypeDescriptor type2) = type1 == type2
    (DateDescriptor date1) == (DateDescriptor date2) = date1 == date2
    (TimeDescriptor time1) == (TimeDescriptor time2) = time1 == time2
    (ParticipantsDescriptor participants1) == (ParticipantsDescriptor participants2) = 
        participants1 == participants2
    (RepetitionsDescriptor repetitions1) == (RepetitionsDescriptor repetitions2) = 
        repetitions1 == repetitions2
    _ == _ = False

## Event display

> 2. A function to show the events in a pretty way, following the order of descriptors as presented above.

Because the `Event` type is just a list of `Descriptor`s with no particular order, we need to sort it before printing it.

For this, we created a set of "getter" functions that find instances of a given descriptor in a list of descriptors.

In [9]:
getName :: Event -> Name
getName event = case find isNameDescriptor event of
        Just (NameDescriptor name) -> name
        _ -> error "Event has no name"
    where
        isNameDescriptor (NameDescriptor _) = True
        isNameDescriptor _ = False

getType :: Event -> Type
getType event = case find isTypeDescriptor event of
        Just (TypeDescriptor type') -> type'
        _ -> error "Event has no type"
    where
        isTypeDescriptor (TypeDescriptor _) = True
        isTypeDescriptor _ = False

getDate :: Event -> Maybe Date
getDate event = case find isDateDescriptor event of
        Just (DateDescriptor date) -> Just date
        _ -> Nothing
    where
        isDateDescriptor (DateDescriptor _) = True
        isDateDescriptor _ = False

getTime :: Event -> Time
getTime event = case find isTimeDescriptor event of
        Just (TimeDescriptor time) -> time
        _ -> error "Event has no time"
    where
        isTimeDescriptor (TimeDescriptor _) = True
        isTimeDescriptor _ = False

getParticipants :: Event -> Participants
getParticipants event = case find isParticipantsDescriptor event of
        Just (ParticipantsDescriptor participants) -> participants
        _ -> error "Event has no participants"
    where
        isParticipantsDescriptor (ParticipantsDescriptor _) = True
        isParticipantsDescriptor _ = False

getRepetitions :: Event -> Repetitions
getRepetitions event = case find isRepetitionsDescriptor event of
        Just (RepetitionsDescriptor repetitions) -> repetitions
        _ -> error "Event has no repetitions descriptor"
    where
        isRepetitionsDescriptor (RepetitionsDescriptor _) = True
        isRepetitionsDescriptor _ = False

In [10]:
printEvent :: Event -> IO ()
printEvent event = do
    putStrLn "Event:"
    putStrLn $ "  Name: " ++ show (getName event)
    putStrLn $ "  Type: " ++ show (getType event)
    putStrLn $ "  Date: " ++ maybe "Unknown" show (getDate event)
    putStrLn $ "  Time: " ++ show (getTime event)
    putStrLn $ "  Participants: " ++ show (getParticipants event)
    putStrLn $ "  Repetitions: " ++ show (getRepetitions event)

-- Example usage
eventName = NameDescriptor "Example"
eventType = TypeDescriptor Work
eventDate = DateDescriptor (Date 6 "December" 2023 True)
eventTime = TimeDescriptor (Time (Just (9, 0)) (Just (10, 30)) Nothing)
eventParticipants = ParticipantsDescriptor (Participants ["John", "Alice", "Bob"])
eventRepetitions = RepetitionsDescriptor (Weekly 2)
exampleEvent = [eventName, eventType, eventDate, eventTime, eventParticipants, eventRepetitions]
printEvent exampleEvent

Event:
  Name: "Example"
  Type: Work
  Date: 6 December 2023 (Workday)
  Time: 09:00 - 10:30 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

## Utility functions

> 3. An `isWorkingDate Date [Date]` function that receives a `Date` and a list of holidays and returns if the day is weekday or weekend/holiday by considering leap years and the fact that 1<sup>st</sup> January 2022 was Saturday.

For this function, we decided to implement the Doomsday algorithm, which allows to compute the day of the week of any given date.

For this century, the anchor day must be a Tuesday, so we chose the 4<sup>th</sup> of January of 2022. Computation for different centuries would be easy to implement (all we need is to change the anchor day) but we assume for now we are only interested in scheduling for the 21<sup>st</sup> century.

In [11]:
calculateDoomsday :: Date -> Int
calculateDoomsday dt@(Date d m y w) = mod s 7
    where
        s = step1 + step2 + step3 + anchorday
        step1 = div (mod y 2000) 12    -- times the number 12 fit as a whole into the two last digits of the year number
        step2 = mod y 2000 - (step1 * 12)    -- difference between the two last digits of the year number and the product of the multiples of 12 from step1
        step3 = div step2 4    -- times the number 4 fit into the result of step2
        anchorday = 2    -- Use as anchor day the 4th of January of 2022 (tuesday)

Once we have found out which day of the week is the Doomsday of the year asked, we need to generate a list with all this Doomsdays. That is, the days of the given year which fall for certain in the calculated day of the week.

In [12]:
generateDoomsdays :: Date -> [Date]
generateDoomsdays dt@(Date d m y w)
    | isleap dt = [Date 4 "January" y w, Date 29 "February" y w, Date 7 "March" y w, Date 4 "April" y w, Date 9 "May" y w, Date 6 "June" y w, Date 11 "July" y w, Date 8 "August" y w, Date 5 "September" y w, Date 10 "October" y w, Date 7 "November" y w, Date 12 "December" y w]
    | otherwise = [Date 3 "January" y w, Date 28 "February" y w, Date 7 "March" y w, Date 4 "April" y w, Date 9 "May" y w, Date 6 "June" y w, Date 11 "July" y w, Date 8 "August" y w, Date 5 "September" y w, Date 10 "October" y w, Date 7 "November" y w, Date 12 "December" y w]

The next step is to implement a function which calculates how many days fall between the date we are seeking and the closest Doomsday. In order to do this, we implemented the function recursively.

In [13]:
doomsday :: Date -> Int
doomsday dt
    | dt `elem` dd = calculateDoomsday dt
    | otherwise = itdoomsday 1 (succ dt)
    where dd = generateDoomsdays dt

itdoomsday :: Int -> Date -> Int
itdoomsday i dt
    | dt `elem` dd = calculateDoomsday dt - i
    | otherwise = itdoomsday (i+1) (succ dt)
    where dd = generateDoomsdays dt

Finally, we can check if the date is a holiday, Saturday (6) or Sunday (0), and if it is, return `False`.

In [14]:
isWorkingDate :: Date -> [Date] -> Bool
isWorkingDate date holidays
    | date `elem` holidays = False
    | mod (doomsday date) 7 == 6 = False
    | mod (doomsday date) 7 == 0 = False
    | otherwise = True

In [15]:
isWorkingDate (Date 12 "December" 2023 True) [Date 25 "December" 2023 True]
isWorkingDate (Date 10 "December" 2023 True) [Date 25 "December" 2023 True]
isWorkingDate (Date 25 "December" 2023 True) [Date 25 "December" 2023 True]
isWorkingDate (Date 9 "December" 2023 True) [Date 25 "December" 2023 True]
isWorkingDate (Date 23 "February" 2024 True) [Date 25 "December" 2023 True]

True

False

False

False

True

> 4. A `searchEvents [Descriptor] Agenda` function that returns all the events of the `Agenda` with such list of `Descriptor`s. It should be able for example to return  events  that  start  at  a  given  time,  with  a  given  frequency,  involving  some persons, etc.

A simple `filter` function can be used to get all the events that match a given list of descriptors.

In [16]:
searchEvents :: [Descriptor] -> Agenda -> [Event]
searchEvents descriptors = filter (matches descriptors)
    where
        matches :: [Descriptor] -> Event -> Bool
        matches descriptors event = all (`elem` event) descriptors

This is not requested, but a function that prints the results in a fancy way is useful.

In [17]:
printEventList :: [Event] -> IO ()
printEventList events = do
    putStrLn $ "Found " ++ show (length events) ++ " events:"
    mapM_ printEvent events

-- Example usage
exampleAgenda = [[NameDescriptor "Event 1", TypeDescriptor Health, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (10, 30)) (Just (12, 0)) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)],
                    [NameDescriptor "Event 2", TypeDescriptor Work, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (9, 0)) (Just (10, 30)) Nothing), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)],
                    [NameDescriptor "Event 3", TypeDescriptor Work, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (9, 0)) (Just (10, 30)) Nothing), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)],
                    [NameDescriptor "Event 4", TypeDescriptor Work, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (9, 0)) (Just (10, 30)) Nothing), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]]
result = searchEvents [TypeDescriptor Work] exampleAgenda
printEventList result

Found 3 events:
Event:
  Name: "Event 2"
  Type: Work
  Date: 1 January 2023 (Workday)
  Time: 09:00 - 10:30 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 3"
  Type: Work
  Date: 1 January 2023 (Workday)
  Time: 09:00 - 10:30 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 4"
  Type: Work
  Date: 1 January 2023 (Workday)
  Time: 09:00 - 10:30 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

> 5. A `deleteEvents [Descriptor] Agenda` function that returns an `Agenda` where the events having those descriptors are removed.

Again, using `filter` it's possible to search for events that match a `Descriptor` list.
The `matches` function checks if all the descriptors are present in the event.

In [18]:
deleteEvents :: [Descriptor] -> Agenda -> Agenda
deleteEvents descriptors = filter (not . matches descriptors)
    where
        matches :: [Descriptor] -> Event -> Bool
        matches descriptors event = all (`elem` event) descriptors

-- Example usage
result = deleteEvents [TypeDescriptor Work] exampleAgenda
printEventList result

Found 1 events:
Event:
  Name: "Event 1"
  Type: Health
  Date: 1 January 2023 (Workday)
  Time: 10:30 - 12:00 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

> 6. A `completeTime Event` function that if the `Event` has not `duration` inserts it by considering the `start` and `end` hours. If it has `start` and duration it will insert the end and vice versa. If it has just duration it will do  nothing, as the `Event` has not been scheduled yet. Assume no event will spawn over different days.

This function requires a bit of logic to work properly.  
There are three cases to consider:

1. The event does not have a duration and it can't be computed. In this case, we return an error.
2. The missing field(s) can be calculated. In this case, we return the event with the missing field(s) filled.
3. The event has all the fields or it only has a duration. In this case, we return the event as it is.

For the calculation of the missing fields, an auxiliary function `completeTime'` is used. This function takes an event and a list of descriptors and returns the event with the missing fields filled. It also contains some error handling, similar to the one that was just described.

If any missing fields were calculated, we remove the old, incomplete `TimeDescriptor` and append the new, complete one. If not, the function returns the event as it is.

In [19]:
completeTime :: Event -> Event
completeTime event = case getTime event of
        time@(Time Nothing Nothing Nothing) -> error "Invalid time"
        time@(Time (Just (startHour, startMinute)) Nothing Nothing) -> error "Invalid time"
        time@(Time Nothing (Just (endHour, endMinute)) Nothing) -> error "Invalid time"
        time@(Time (Just (startHour, startMinute)) (Just (endHour, endMinute)) Nothing) -> TimeDescriptor (completeTime' time) : cleanEvent time
        time@(Time (Just (startHour, startMinute)) Nothing (Just duration)) -> TimeDescriptor (completeTime' time) : cleanEvent time
        time@(Time Nothing (Just (endHour, endMinute)) (Just duration)) -> TimeDescriptor (completeTime' time) : cleanEvent time
        time@(Time Nothing Nothing (Just duration)) -> event
        time@(Time (Just (startHour, startMinute)) (Just (endHour, endMinute)) (Just duration)) -> event
        _ -> event
        where
                cleanEvent time = filter (/= TimeDescriptor time) event
                completeTime' :: Time -> Time
                completeTime' (Time Nothing Nothing Nothing) = error "Invalid time"
                completeTime' (Time (Just (startHour, startMinute)) Nothing Nothing) = error "Invalid time"
                completeTime' (Time Nothing (Just (endHour, endMinute)) Nothing) = error "Invalid time"
                completeTime' (Time Nothing Nothing (Just duration)) = Time Nothing Nothing (Just duration)
                completeTime' (Time (Just (startHour, startMinute)) (Just (endHour, endMinute)) Nothing) =
                    Time (Just (startHour, startMinute)) (Just (endHour, endMinute)) (Just ((endHour * 60 + endMinute) - (startHour * 60 + startMinute)))
                completeTime' (Time (Just (startHour, startMinute)) Nothing (Just duration)) =
                        Time (Just (startHour, startMinute)) (Just (startHour + (startMinute + duration) `div` 60, (startMinute + duration) `mod` 60)) (Just duration)
                completeTime' (Time Nothing (Just (endHour, endMinute)) (Just duration)) =
                        Time (Just (endHour - (endMinute - duration) `div` 60, (endMinute - duration) `mod` 60)) (Just (endHour, endMinute)) (Just duration)
                completeTime' (Time (Just (startHour, startMinute)) (Just (endHour, endMinute)) (Just duration)) =
                        Time (Just (startHour, startMinute)) (Just (endHour, endMinute)) (Just duration)

-- Example usage
exampleEvent = [NameDescriptor "Event 1", TypeDescriptor Health, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (10, 30)) (Just (12, 0)) Nothing), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]
printEvent exampleEvent
printEvent $ completeTime exampleEvent

Event:
  Name: "Event 1"
  Type: Health
  Date: 1 January 2023 (Workday)
  Time: 10:30 - 12:00 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

Event:
  Name: "Event 1"
  Type: Health
  Date: 1 January 2023 (Workday)
  Time: 10:30 - 12:00 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

> 7. A `listAgenda Agenda` function that returns all the `Events` of an `Agenda` sorted from earliest `Date` and `Time` to latest. Events which are not yet scheduled will be shown at the end and sorted by their `name` in alphabetical order.

In order to implement this function, we will program two auxiliary functions that order a list of events by a given requirement.

Firstly, we will implement a function that orders the events by the date and time of it.

In [20]:
sortAgendaDateTime :: [Event] -> [Event]
sortAgendaDateTime [] = []
sortAgendaDateTime (x:xs) = insert' x (sortAgendaDateTime xs)
    where
        insert' :: Event -> [Event] -> [Event]
        insert' event [] = [event]
        insert' event (x:xs) 
            | getDate event < getDate x = event : x : xs
            | getDate event == getDate x && getTime event < getTime x = event : x : xs
            | otherwise = x : insert' event xs

The next step is to implement a function that orders a list of events by the name in alphabetical order.

In [21]:
sortAgendaName :: [Event] -> [Event]
sortAgendaName [] = []
sortAgendaName (x:xs) = insert' x (sortAgendaName xs)
    where
        insert' :: Event -> [Event] -> [Event]
        insert' event [] = [event]
        insert' event (x:xs) 
            | getName event < getName x = event : x : xs
            | otherwise = x : insert' event xs

Finally, we use this two functions to order the complete agenda. We separate the lists and give to each auxiliary function only the events it has to order. By concatenating both lists, we get the list of events order as we were looking for.

In [22]:
listAgenda :: Agenda -> [Event]
listAgenda agenda = sortAgendaDateTime (scheduledEvents agenda) ++ sortAgendaName (unscheduledEvents agenda)
    where
        scheduledEvents :: Agenda -> [Event]
        scheduledEvents [] = []
        scheduledEvents (x:xs)
            | getDate x /= Nothing && not (onlyHasDuration (getTime x)) = x : scheduledEvents xs 
            | otherwise = scheduledEvents xs
        unscheduledEvents :: Agenda -> [Event]
        unscheduledEvents [] = []
        unscheduledEvents (x:xs)
            | getDate x == Nothing || onlyHasDuration (getTime x) = x : unscheduledEvents xs 
            | otherwise = unscheduledEvents xs

onlyHasDuration :: Time -> Bool
onlyHasDuration (Time Nothing Nothing (Just duration)) = True
onlyHasDuration _ = False

Line 7: Use isJust
Found:
getDate x /= Nothing
Why not:
Data.Maybe.isJust (getDate x)Line 12: Use isNothing
Found:
getDate x == Nothing
Why not:
isNothing (getDate x)

In [23]:
-- Example usage
exampleAgenda = [[NameDescriptor "aEvent 1", TypeDescriptor Health, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (10, 30)) (Just (12, 0)) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)],
                    [NameDescriptor "dEvent 2", TypeDescriptor Work, DateDescriptor (Date 3 "January" 2023 True), TimeDescriptor (Time (Just (9, 0)) (Just (10, 30)) Nothing), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)],
                    [NameDescriptor "bEvent 3", TypeDescriptor Work, TimeDescriptor (Time (Just (10, 50)) (Just (11, 30)) Nothing), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)],
                    [NameDescriptor "cEvent 4", TypeDescriptor Work, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (9, 0)) (Just (10, 30)) Nothing), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]]

listAgenda exampleAgenda

[[NameDescriptor "cEvent 4",TypeDescriptor Work,DateDescriptor 1 January 2023 (Workday),TimeDescriptor 09:00 - 10:30 (90 minutes),ParticipantsDescriptor John, Alice, Bob,RepetitionsDescriptor Weekly 2],[NameDescriptor "aEvent 1",TypeDescriptor Health,DateDescriptor 1 January 2023 (Workday),TimeDescriptor 10:30 - 12:00 (90 minutes),ParticipantsDescriptor John, Alice, Bob,RepetitionsDescriptor Weekly 2],[NameDescriptor "dEvent 2",TypeDescriptor Work,DateDescriptor 3 January 2023 (Workday),TimeDescriptor 09:00 - 10:30 (90 minutes),ParticipantsDescriptor John, Alice, Bob,RepetitionsDescriptor Weekly 2],[NameDescriptor "bEvent 3",TypeDescriptor Work,TimeDescriptor 10:50 - 11:30 (40 minutes),ParticipantsDescriptor John, Alice, Bob,RepetitionsDescriptor Weekly 2]]

> 8. An `insert Event Agenda` function that for events fully specified in terms of `Date` and `Time` returns the `Agenda` with the `Event` included if it does not overlap with any other. If it overlaps  the original `Agenda` will be returned. If the `Event` is not fully specified, it will raise an error. 

First let us add some utility functions for determining if an `Event` is fully specified and if two `Event`s overlap.

In [24]:
isTimeSpecified :: Event -> Bool
isTimeSpecified event = case getTime event of
        time@(Time (Just (startHour, startMinute)) (Just (endHour, endMinute)) (Just duration)) -> True
        _ -> False

isDateSpecified :: Event -> Bool
isDateSpecified event = case getDate event of
        Just date -> True
        _ -> False

isFullySpecified :: Event -> Bool
isFullySpecified event = isDateSpecified event && isTimeSpecified event

In [71]:
overlaps :: Event -> Event -> Bool
overlaps event1 event2
    | not (isFullySpecified event1) || not (isFullySpecified event2) = True
    | getDate event1 /= getDate event2 = False
    | otherwise = (startHour1, startMinute1) <= (endHour2, endMinute2) && (endHour1, endMinute1) >= (startHour2, startMinute2)
    where
        (startHour1, startMinute1) = fromJust $ start (getTime event1)
        (endHour1, endMinute1) = fromJust $ end (getTime event1)
        (startHour2, startMinute2) = fromJust $ start (getTime event2)
        (endHour2, endMinute2) = fromJust $ end (getTime event2) 

-- Example usage
event1 = [NameDescriptor "Event 1", TypeDescriptor Health, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (10, 30)) (Just (12, 30)) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]
event2 = [NameDescriptor "Event 2", TypeDescriptor Health, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (19, 30)) (Just (21, 0)) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]
event3 = [NameDescriptor "Event 3", TypeDescriptor Health, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (12, 10)) (Just (21, 0)) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]
event1 `overlaps` event2
event1 `overlaps` event3
event2 `overlaps` event3

event4 = [NameDescriptor "Event 4", TypeDescriptor Health, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (9, 0)) (Just (12, 0)) (Just 180)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]
event5 = [NameDescriptor "Event 5", TypeDescriptor Work, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (9, 0)) (Just (10, 30)) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]
event5 `overlaps` event4


False

True

True

True

The `insert` function is then implemented to check if the `Event` is fully specified and if it overlaps with any other `Event` in the `Agenda`. If it is not fully specified or it overlaps, the original `Agenda` is returned. Otherwise, the `Event` is inserted in the `Agenda`.

In [72]:
insert :: Event -> Agenda -> Agenda
insert event agenda
    | not (isFullySpecified event) = error "Event is not fully specified"
    | any (overlaps event) agenda = agenda
    | otherwise = event : agenda

-- Example usage
exampleAgenda = [[NameDescriptor "Event 1", TypeDescriptor Health, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (10, 30)) (Just (12, 0)) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)],
                    [NameDescriptor "Event 2", TypeDescriptor Work, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (9, 0)) (Just (10, 30)) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]]
exampleEvent = [NameDescriptor "Event 5", TypeDescriptor Work, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (19, 0)) (Just (20, 30)) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]
printEventList $ insert exampleEvent exampleAgenda

Found 3 events:
Event:
  Name: "Event 5"
  Type: Work
  Date: 1 January 2023 (Workday)
  Time: 19:00 - 20:30 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 1"
  Type: Health
  Date: 1 January 2023 (Workday)
  Time: 10:30 - 12:00 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 2"
  Type: Work
  Date: 1 January 2023 (Workday)
  Time: 09:00 - 10:30 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

## Current

> 9. A `scheduleEvent` function that inserts an `Event` with no `Date` in the `Agenda` and schedules it as earliest as possible between `StartDate` and `EndDate`. If the `Event` cannot be scheduled, the original `Agenda` will be returned. Notice that the event can be partially or fully defined in terms of `Time`. Do not consider repetitions.

The logic for this functionality is quite complex, so let us break it down into smaller parts.

Let's start with a function that tells us if, given a `Time` and `Date`, that time slot is available in the `Agenda`.

In [73]:
isTimeSlotAvailable :: Date -> Time -> Agenda -> Bool
isTimeSlotAvailable date time agenda = not (any (overlaps event) agenda)
  where
    event = [TimeDescriptor time, DateDescriptor date]

exampleAgenda = [[NameDescriptor "Event 1", TypeDescriptor Health, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (10, 30)) (Just (12, 0)) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)],
                    [NameDescriptor "Event 2", TypeDescriptor Work, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (9, 0)) (Just (10, 30)) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)],
                    [NameDescriptor "Event 3", TypeDescriptor Work, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (0, 0)) (Just (1, 0)) (Just 60)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]]
isTimeSlotAvailable (Date 1 "January" 2023 True) (Time (Just (9, 0)) (Just (12, 0)) (Just 180)) exampleAgenda
isTimeSlotAvailable (Date 1 "January" 2023 True) (Time (Just (10, 30)) (Just (12, 0)) (Just 90)) exampleAgenda
isTimeSlotAvailable (Date 1 "January" 2023 True) (Time (Just (12, 30)) (Just (14, 0)) (Just 90)) exampleAgenda

False

False

True

This function is useful for trying dates until we find one that works.  
The next step is to implement a recursive function that tries to schedule an event in a given date. If it can't, it tries the next day and so on until it finds a date that works or it runs out of days.

In [74]:
tryingDates :: Event -> Agenda -> Date -> Date -> Agenda
tryingDates event agenda startDate endDate
  | startDate > endDate = agenda
  | isTimeSlotAvailable startDate (getTime event) agenda = c_event : agenda
  | otherwise = tryingDates event agenda (succ startDate) endDate
  where
    c_event = [NameDescriptor (getName event), TypeDescriptor (getType event), DateDescriptor startDate, TimeDescriptor (getTime event), ParticipantsDescriptor (getParticipants event), RepetitionsDescriptor (getRepetitions event)]

-- Example usage
exampleEvent = [NameDescriptor "Event 5", TypeDescriptor Work, TimeDescriptor (Time (Just (9, 0)) (Just (12, 0)) (Just 180)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]
printEventList $ sortAgendaDateTime $ tryingDates exampleEvent exampleAgenda (Date 1 "January" 2023 True) (Date 5 "January" 2023 True)


Found 4 events:
Event:
  Name: "Event 3"
  Type: Work
  Date: 1 January 2023 (Workday)
  Time: 00:00 - 01:00 (60 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 2"
  Type: Work
  Date: 1 January 2023 (Workday)
  Time: 09:00 - 10:30 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 1"
  Type: Health
  Date: 1 January 2023 (Workday)
  Time: 10:30 - 12:00 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 5"
  Type: Work
  Date: 2 January 2023 (Workday)
  Time: 09:00 - 12:00 (180 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

We can also implement a function similar to tryingDates, but with times instead of dates.  
One could try moving the timeslot required for a new event one minute at a time, similar to how we moved the date one day at a time.

In [75]:
moveOneMinute :: Time -> Time
moveOneMinute (Time (Just (startHour, startMinute)) (Just (endHour, endMinute)) (Just duration)) =
  let (newStartHour, newStartMinute) = if startMinute + 1 > 59 then (startHour + 1, 0) else (startHour, startMinute + 1)
      (newEndHour, newEndMinute) = if endMinute + 1 > 59 then (endHour + 1, 0) else (endHour, endMinute + 1)
  in Time (Just (newStartHour, newStartMinute)) (Just (newEndHour, newEndMinute)) (Just duration)

-- Example usage
moveOneMinute (Time (Just (8, 59)) (Just (11, 59)) (Just 180))

moveEventOneMinute :: Event -> Event
moveEventOneMinute event = case getTime event of
        time@(Time Nothing Nothing Nothing) -> error "Invalid time"
        time@(Time (Just (startHour, startMinute)) Nothing Nothing) -> error "Invalid time"
        time@(Time Nothing (Just (endHour, endMinute)) Nothing) -> error "Invalid time"
        time@(Time (Just (startHour, startMinute)) (Just (endHour, endMinute)) Nothing) -> TimeDescriptor (moveOneMinute time) : cleanEvent time
        time@(Time (Just (startHour, startMinute)) Nothing (Just duration)) -> TimeDescriptor (moveOneMinute time) : cleanEvent time
        time@(Time Nothing (Just (endHour, endMinute)) (Just duration)) -> TimeDescriptor (moveOneMinute time) : cleanEvent time
        time@(Time Nothing Nothing (Just duration)) -> event
        time@(Time (Just (startHour, startMinute)) (Just (endHour, endMinute)) (Just duration)) -> TimeDescriptor (moveOneMinute time) : cleanEvent time
        _ -> event
        where
                cleanEvent time = filter (/= TimeDescriptor time) event
-- Example usage
exampleEvent = [NameDescriptor "Event 1", TypeDescriptor Health, DateDescriptor (Date 1 "January" 2023 True), TimeDescriptor (Time (Just (10, 30)) (Just (12, 0)) (Just 90)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]
printEvent $ moveEventOneMinute exampleEvent

09:00 - 12:00 (180 minutes)

Event:
  Name: "Event 1"
  Type: Health
  Date: 1 January 2023 (Workday)
  Time: 10:31 - 12:01 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

In [76]:
tryingTimes :: Event -> Agenda -> Date -> Date -> Agenda
tryingTimes event agenda startDate endDate
  | startDate > endDate = agenda
  | isTimeSlotAvailable startDate (getTime def_event) agenda = c_event : agenda
  | otherwise = tryingTimes' min_event agenda startDate endDate
  where
    def_event = setDefaultStartTime event
    c_event = completeTime def_event
    min_event = moveEventOneMinute c_event
    tryingTimes' :: Event -> Agenda -> Date -> Date -> Agenda
    tryingTimes' event agenda startDate endDate
      | startDate > endDate = agenda
      | getTime c_event > Time (Just (23, 59)) (Just (23, 59)) Nothing = trace (show c_event) $ tryingTimes event agenda (succ startDate) endDate
      | isTimeSlotAvailable startDate (getTime event) agenda = c_event : agenda
      | otherwise = tryingTimes' min_event agenda startDate endDate
      where
        min_event = moveEventOneMinute event
        c_event = completeTime event

setDefaultStartTime :: Event -> Event
setDefaultStartTime event = case getTime event of
  Time _ _ (Just duration) -> TimeDescriptor (Time (Just (0, 0)) Nothing (Just duration)) : cleanEvent
  _ -> error "Invalid time"
  where
    cleanEvent = filter (/= TimeDescriptor (getTime event)) event

In [77]:
scheduleEvent :: Event -> Agenda -> Date -> Date -> Agenda
scheduleEvent event agenda startDate endDate
  | -- if the event has a start time, we only have to find a date
    isTimeSpecified event = tryingDates event agenda startDate endDate
  | -- if the event has a duration, we have to find a date and a start time
    onlyHasDuration (getTime event) = tryingTimes event agenda startDate endDate

In [79]:
-- Example usage
exampleEventNoDate = [NameDescriptor "Event 5", TypeDescriptor Work, TimeDescriptor (Time (Just (9, 0)) (Just (12, 0)) (Just 180)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2)]
printEventList $ sortAgendaDateTime $ scheduleEvent exampleEventNoDate exampleAgenda (Date 1 "January" 2023 True) (Date 5 "January" 2023 True)
exampleEventNoTime = [NameDescriptor "Event 5", TypeDescriptor Work, TimeDescriptor (Time Nothing Nothing (Just 180)), ParticipantsDescriptor (Participants ["John", "Alice", "Bob"]), RepetitionsDescriptor (Weekly 2), DateDescriptor (Date 1 "January" 2023 True)]
printEventList $ sortAgendaDateTime $ scheduleEvent exampleEventNoTime exampleAgenda (Date 1 "January" 2023 True) (Date 5 "January" 2023 True)

Found 4 events:
Event:
  Name: "Event 3"
  Type: Work
  Date: 1 January 2023 (Workday)
  Time: 00:00 - 01:00 (60 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 2"
  Type: Work
  Date: 1 January 2023 (Workday)
  Time: 09:00 - 10:30 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 1"
  Type: Health
  Date: 1 January 2023 (Workday)
  Time: 10:30 - 12:00 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 5"
  Type: Work
  Date: 2 January 2023 (Workday)
  Time: 09:00 - 12:00 (180 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

Found 4 events:
Event:
  Name: "Event 3"
  Type: Work
  Date: 1 January 2023 (Workday)
  Time: 00:00 - 01:00 (60 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 2"
  Type: Work
  Date: 1 January 2023 (Workday)
  Time: 09:00 - 10:30 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 5"
  Type: Work
  Date: 1 January 2023 (Workday)
  Time: 01:01 - 04:01 (180 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2
Event:
  Name: "Event 1"
  Type: Health
  Date: 1 January 2023 (Workday)
  Time: 10:30 - 12:00 (90 minutes)
  Participants: John, Alice, Bob
  Repetitions: Weekly 2

> 10. An `allPossibleSchedules` function that takes an `Event`, an `Agenda`, a `StartDate`, and an `EndDate` as input and returns a list of `Agendas` with all the possible slots where the `Event` can be scheduled. This function does not consider repetitions.